# Reverse Geo Big Cities

Provides a lat/long sorted list with the name of the largest city occupying that space, restricted to cities with population over 15,000. This tool makes it easier to read charts with gridded weather data by providing a convenient geographical reference.  It will be used in the "2018 weather" web site, being built by me, so I am the logical choice for maintainer.  This data is needed before the web site can go live.  
By:  Andrew Guenthner
v0.5: 7-May-2019

### Special Thanks
GeoMap.org (www.geomap.org) kindly provided the initial list of cities over 15,000 as a text file, which I have converted to .csv.  The data was provided under the Creative Commons 4.0 license.
The ISO country codes list was provided by the International Standards Organization web site.

## Requirements

*  Input file named 'cities15000.csv' in a folder called 'Geo' in the same folder as this notebook.
*  Country codes file named 'iso_country_codes.csv' also in the 'Geo' folder
*  Pandas (0.23 or higher)

In [1]:
#Import dependencies and files
import pandas as pd
import datetime as dt

srcfile = 'Geo/cities15000.csv'
codefile = 'Geo/iso_country_codes.csv'

In [2]:
#Read data -- raw format
city_df = pd.read_csv(srcfile, header=None)
city_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,P,PPLA,AD,NaN,8,NaN,NaN,NaN,15853,NaN,1033,Europe/Andorra,10/15/2008
1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,P,PPLC,AD,NaN,7,NaN,NaN,NaN,20430,NaN,1037,Europe/Andorra,5/30/2010
2,290594,Umm al Qaywayn,Umm al Qaywayn,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",25.56473,55.55517,P,PPLA,AE,NaN,7,NaN,NaN,NaN,44411,NaN,2,Asia/Dubai,10/7/2014
3,291074,Ras al-Khaimah,Ras al-Khaimah,"Julfa,Khaimah,RAK City,RKT,Ra's al Khaymah,Ra'...",25.78953,55.94320,P,PPLA,AE,NaN,5,NaN,NaN,NaN,115949,NaN,2,Asia/Dubai,5/6/2019
4,291696,Khawr Fakkān,Khawr Fakkan,"Fakkan,Fakkān,Khawr Fakkan,Khawr Fakkān,Khawr ...",25.33132,56.34199,P,PPL,AE,NaN,6,NaN,NaN,NaN,33575,NaN,20,Asia/Dubai,10/25/2013


### Pre-process Data

In [3]:
# Drop the columns we do not need
city_df = city_df.drop(columns = [0,1,3,6,7,9,10,11,12,13,15,16,17,18])
# Rename the remaining columsn to something easier to read
city_df.columns = ['city','lat','long','ctry_code','pop']
city_df.head()

,city,lat,long,ctry_code,pop
0,les Escaldes,42.50729,1.53414,AD,15853
1,Andorra la Vella,42.50779,1.52109,AD,20430
2,Umm al Qaywayn,25.56473,55.55517,AE,44411
3,Ras al-Khaimah,25.78953,55.94320,AE,115949
4,Khawr Fakkan,25.33132,56.34199,AE,33575


In [4]:
# We need to fix the longitude so that it matches our weather data.  
# In the display sytem, longitudes west of the prime meridian are negative,
# we need them to be between 180 and 360 degrees East to match the weather data.
city_df['new_long'] = city_df['long'].apply(lambda x: (360.0 + x) if (x < 0) else x)
city_df.head()

,city,lat,long,ctry_code,pop,new_long
0,les Escaldes,42.50729,1.53414,AD,15853,1.53414
1,Andorra la Vella,42.50779,1.52109,AD,20430,1.52109
2,Umm al Qaywayn,25.56473,55.55517,AE,44411,55.55517
3,Ras al-Khaimah,25.78953,55.94320,AE,115949,55.94320
4,Khawr Fakkan,25.33132,56.34199,AE,33575,56.34199


In [5]:
# Now replace the country code with a country name by joining
# First, make a country names DataFrame
country_df = pd.read_csv(codefile)
country_df.head()

,English short name,French short name,Alpha-2 code,Alpha-3 code,Numeric
0,Andorra,Andorre (l'),AD,AND,20
1,United Arab Emirates (the),Émirats arabes unis (les),AE,ARE,784
2,Afghanistan,Afghanistan (l'),AF,AFG,4
3,Antigua and Barbuda,Antigua-et-Barbuda,AG,ATG,28
4,Anguilla,Anguilla,AI,AIA,660


In [6]:
# Trim this DataFrame to have only what we need (English name and code)
country_df = country_df[['English short name','Alpha-2 code']]
# Rename the columns to prepare for merge
country_df.columns = ['country','ctry_code']
country_df.head()

,country,ctry_code
0,Andorra,AD
1,United Arab Emirates (the),AE
2,Afghanistan,AF
3,Antigua and Barbuda,AG
4,Anguilla,AI


In [7]:
# Merge the country name in
city_df = city_df.merge(country_df,on='ctry_code')
city_df.head()

,city,lat,long,ctry_code,pop,new_long,country
0,les Escaldes,42.50729,1.53414,AD,15853,1.53414,Andorra
1,Andorra la Vella,42.50779,1.52109,AD,20430,1.52109,Andorra
2,Umm al Qaywayn,25.56473,55.55517,AE,44411,55.55517,United Arab Emirates (the)
3,Ras al-Khaimah,25.78953,55.94320,AE,115949,55.94320,United Arab Emirates (the)
4,Khawr Fakkan,25.33132,56.34199,AE,33575,56.34199,United Arab Emirates (the)


In [8]:
# Clean up the data frame by dropping the old long and renaming the new_long to long
city_df = city_df.drop(columns='long')
city_df = city_df.rename(columns = {'new_long':'long'})
city_df.head()

,city,lat,ctry_code,pop,long,country
0,les Escaldes,42.50729,AD,15853,1.53414,Andorra
1,Andorra la Vella,42.50779,AD,20430,1.52109,Andorra
2,Umm al Qaywayn,25.56473,AE,44411,55.55517,United Arab Emirates (the)
3,Ras al-Khaimah,25.78953,AE,115949,55.94320,United Arab Emirates (the)
4,Khawr Fakkan,25.33132,AE,33575,56.34199,United Arab Emirates (the)


In [9]:
#Rearrange to make a more logical DataFrame
neworder = ['lat','long','city','country','pop','ctry_code']
city_df = city_df.reindex(columns=neworder)
#Drop the country code
city_df = city_df.drop(columns='ctry_code')
city_df.head()

,lat,long,city,country,pop
0,42.50729,1.53414,les Escaldes,Andorra,15853
1,42.50779,1.52109,Andorra la Vella,Andorra,20430
2,25.56473,55.55517,Umm al Qaywayn,United Arab Emirates (the),44411
3,25.78953,55.94320,Ras al-Khaimah,United Arab Emirates (the),115949
4,25.33132,56.34199,Khawr Fakkan,United Arab Emirates (the),33575


### Organize the Cities by Lat / Lon and Sort by Population

In [10]:
city_df.iloc[200:220,]

,lat,long,city,country,pop
200,-34.64966,301.61659,Barracas,Argentina,77474
201,-36.77698,300.14146,Azul,Argentina,53941
202,-29.11761,300.34166,Avellaneda,Argentina,23077
203,-27.09625,305.10374,Aristobulo del Valle,Argentina,20683
204,-38.89916,289.94558,Zapala,Argentina,31534
205,-26.81667,294.68333,Yerba Buena,Argentina,50783
206,-39.10000,292.93333,Villa Regina,Argentina,31209
207,-31.49330,291.46162,Villa Paula de Sarmiento,Argentina,19092
208,-32.43293,296.75237,Villa Nueva,Argentina,16841
209,-32.40751,296.75984,Villa Maria,Argentina,92453


In [11]:
# Use pd.cut to assign categories to lat and long values in 3-degree increments
# This will match the gridding on the weather data to be generated
latbins = range(-90,93,3)
latlabels = [59 - n for n in range(60)]
longbins = range(0,363,3)
longlabels = range(120)
city_df['lat_label'] = pd.cut(city_df['lat'], bins=latbins, labels=latlabels)
city_df['long_label'] = pd.cut(city_df['long'], bins=longbins, labels=longlabels,include_lowest = True)
city_df.head()

,lat,long,city,country,pop,lat_label,long_label
0,42.50729,1.53414,les Escaldes,Andorra,15853,15,0
1,42.50779,1.52109,Andorra la Vella,Andorra,20430,15,0
2,25.56473,55.55517,Umm al Qaywayn,United Arab Emirates (the),44411,21,18
3,25.78953,55.94320,Ras al-Khaimah,United Arab Emirates (the),115949,21,18
4,25.33132,56.34199,Khawr Fakkan,United Arab Emirates (the),33575,21,18


In [12]:
# Use a groupby with an internal sort to get a city list by lat/lon group, sorted
# in descending order of population.  
citygroup = city_df.groupby(['lat_label','long_label']).apply(pd.DataFrame.sort_values,
                                                    'pop', ascending=False)
# Do some housekeeping to eliminate the multi-index
citygroup = citygroup.rename(columns={'lat_label':'lat_label1','long_label':'long_label1'})
citygroup = citygroup.reset_index()
citygroup = citygroup.drop(columns = ['level_2','lat_label1','long_label1'])
citygroup.head()

,lat_label,long_label,lat,long,city,country,pop
0,48,97,-54.81084,291.68409,Ushuaia,Argentina,58028
1,48,107,-54.28111,323.49080,Grytviken,South Georgia and the South Sandwich Islands,2
2,47,95,-51.72987,287.49397,Puerto Natales,Chile,20000
3,47,96,-53.15483,289.08871,Punta Arenas,Chile,117430
4,47,96,-51.62261,290.78187,Rio Gallegos,Argentina,85700


In [13]:
# Do a second groupby, in which we retain only the first instance of each city,
# which is the one with the highest population.
largest_citygroup = citygroup.groupby(['lat_label','long_label']).first()
# This gives us every possible combo, which is what we want, but we'll fill NaN with "--"
largest_citygroup = largest_citygroup.fillna('--')
# Sort by lat_label (descending because of how we grouped it earlier), then reset_index
largest_citygroup = largest_citygroup.sort_values(by=['lat_label','long_label'], ascending=[False,True])
largest_citygroup = largest_citygroup.reset_index()
largest_citygroup.head()

,lat_label,long_label,lat,long,city,country,pop
0,0,0,--,--,--,--,--
1,0,1,--,--,--,--,--
2,0,2,--,--,--,--,--
3,0,3,--,--,--,--,--
4,0,4,--,--,--,--,--


In [14]:
# To make sure we did it right, check the length of the largest_citygroup, it should be 7200
len(largest_citygroup)

7200

### Output the Data

In [15]:
# Save to csv
largest_citygroup.to_csv('city_by_latlongroup.csv')

This notebook last executed on:

In [16]:
print(dt.datetime.now())

2019-05-08 01:37:22.746882
